In [44]:
# Импортиурем библиотеку для работы с БД на python
import psycopg2

# Пытаемся подключиться к БД, при ошибке выскакивает предупреждение
try: 
    conn = psycopg2.connect(dbname='postgres', user='postgres', password='123', host='localhost')
    # Получаем объект курсора, для запросов к БД
    cursor = conn.cursor()
except:
    print('Неудалось подключиться к БД!')

In [45]:
# Подаем извлекаем данные запросом из БД
cursor.execute('SELECT name, description, sallary, region_name FROM jobschema.joblist')
# Загружаем данные из запроса и преобрабатываем
all_info = list(map(list, cursor.fetchall()))

In [46]:
# Подаем извлекаем данные запросом из БД
cursor.execute('SELECT skills FROM jobschema.joblist')
# Загружаем данные из запроса и предобрабатываем
all_skills = set(sum([i[0].split(';') for i in list(map(list, cursor.fetchall()))], []))
# Выходим из курсора
cursor.close()
# Отключаемся от БД
conn.close()

In [41]:
# Импортиурем библиотеку для леметизации предложений
import spacy
# Импортируем бибилиотеку для работы со строками
import re

#Загружаем преобученную модель для лемитизации русских слов
nlp = spacy.load("ru_core_news_sm")

# Циклом проходимся по всем описанием из выгруженных данных
for i in range(len(all_info)):
    # Удаляем из описания знаки препинания 
    description = re.sub(r'[.,?:!"-()\';{}]', r'', all_info[i][1])
    # Иницилизируем объект nlp
    doc = nlp(description)
    # Заменяем прошлое описание из выгруженных данных на леметизированное описание
    all_info[i][1] = " ".join([token.lemma_ for token in doc])


In [10]:
import spacy
from collections import Counter

# Загрузка модели языка
nlp = spacy.load("ru_core_news_sm")  # Замените "xx_ent_wiki_sm" на подходящую для вашего языка модель

# Пример описаний вакансий (замените на ваши данные)
job_descriptions = [i[1] for i in all_info]

# Функция для извлечения ключевых слов из текста
def extract_keywords(text):
    doc = nlp(text)
    # Извлечение существительных и глаголов как ключевых слов
    keywords = [token.lemma_ for token in doc if token.pos_ in ["NOUN", "PROPN", "VERB"]]
    return keywords

# Извлечение ключевых слов из описаний вакансий
all_keywords = []
for job_description in job_descriptions:
    keywords = extract_keywords(job_description)
    all_keywords.extend(keywords)

# Подсчет частоты встречаемости ключевых слов
keyword_counts = Counter(all_keywords)

# Вывод наиболее часто встречающихся ключевых слов
top_keywords = keyword_counts.most_common(5)
print("Top keywords:")
for keyword, count in top_keywords:
    print(f"{keyword}: {count}")

Top keywords:
работа: 2145
данных: 1734
-: 1572
опыт: 1447
компания: 902


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np



# Преобразование скиллов в векторное представление с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer()
skills_vectors = tfidf_vectorizer.fit_transform(all_skills)

# Кластеризация скиллов с помощью K-means
num_clusters = 3  # Задайте количество кластеров
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(skills_vectors)

# Получение меток кластеров для каждого скилла
cluster_labels = kmeans.labels_

# Вывод скиллов и их соответствующих кластеров
for skill, label in zip(all_skills, cluster_labels):
    print(f"{skill}: Cluster {label + 1}")

: Cluster 1
oracle business intelligence enterprise edition plus: Cluster 1
работа с low-code инструментами: Cluster 1
microsoft visio: Cluster 3
ведение управленческой отчетности: Cluster 2
etl: Cluster 1
многозадачность: Cluster 1
ведение отчетности: Cluster 2
аналитическое мышление: Cluster 1
ведение финансовой отчетности: Cluster 2
apache kafka: Cluster 1
моделирование рисков: Cluster 1
api: Cluster 1
работа в системах управления проектами: Cluster 1
управление данными: Cluster 1
amazon simple storage service: Cluster 1
работа в команде: Cluster 1
seaborn: Cluster 1
управление ассортиментом: Cluster 1
управление кредитными и процентными рисками: Cluster 1
java/kotlin/scala: Cluster 1
кредитование: Cluster 1
microsoft outlook: Cluster 3
microsoft powerpoint: Cluster 3
бюджетирование: Cluster 1
битрикс: Cluster 1
знание базовых принципов разработки: Cluster 1
экономический анализ: Cluster 1
matlab: Cluster 1
qa / тестирование: Cluster 1
mysql: Cluster 1
обеспечение информационной без